In [1]:
!pip install selenium

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import sys
import urllib.request
import os

import time
import pandas as pd

In [3]:
wd = webdriver.Chrome()

In [6]:
# 해외 판매량

data_list = []

for i in range(2022,2025):
    # 2022 ~ 2024
    for j in range(1,13):
        # 2022-01 ~ 2024-07까지만 출력
        if i == 2024 and j == 8:
            break;
        
        wd.get(f'https://auto.danawa.com/auto/?Work=record&Tab=Grand&Month={i}-{j:02}-00')
        time.sleep(2)
        
        element = wd.find_element(By.CSS_SELECTOR, '#autodanawa_gridC > div.gridMain > article > main > div:nth-child(2) > table > tbody')
        rows = element.find_elements(By.TAG_NAME, 'tr')
        
        sales_month = f'{i}-{j:02}'
        print(sales_month)

        count = 0
        for row in rows:
            count +=1
            if count == 6:
                break
            # 다른 class 속성이 있는 경우 건너뜀 - <tr>에 대해서만 작동
            class_attribute = row.get_attribute('class')
            if class_attribute and ('sub' in class_attribute or 'model_' in class_attribute):
                continue  
            
            # 회사명
            company_element = row.find_element(By.CSS_SELECTOR, 'td.title a')
            company_name = company_element.text.strip()  # 회사명
            
            # 판매량
            sales = row.find_element(By.CLASS_NAME, 'num').text.split(' ')[0].replace(',', '')
            
            # 점유율
            market_share = row.find_element(By.CLASS_NAME, 'rate').text.replace('%', '')
            
            # 전월 대비 판매량 추출
            format_change = lambda text: '-' + text.split('▼')[0].strip() if '▼' in text else text.split('▲')[0].strip()
            last_month_values = row.find_elements(By.CLASS_NAME, 'right')[1].text.split('\n')
            
            last_month_sales = last_month_values[0].replace(',', '') # 전월 판매량
            last_month_change = format_change(last_month_values[1]).replace(',', '') # 전월 대비 증감
            
            data_dict = {
                '회사명': company_name,
                '판매량': sales,
                '점유율': market_share,
                '전월 판매량': last_month_sales,
                '전월 대비 증감': last_month_change,
                '판매월' : sales_month
            }
            print(data_dict)
            data_list.append(data_dict)

2022-01
{'회사명': 'BMW', '판매량': '5550', '점유율': '32.8', '전월 판매량': '4233', '전월 대비 증감': '1317', '판매월': '2022-01'}
{'회사명': '벤츠', '판매량': '3405', '점유율': '20.1', '전월 판매량': '6752', '전월 대비 증감': '-3347', '판매월': '2022-01'}
{'회사명': '아우디', '판매량': '1269', '점유율': '7.5', '전월 판매량': '4373', '전월 대비 증감': '-3104', '판매월': '2022-01'}
{'회사명': '폭스바겐', '판매량': '1213', '점유율': '7.2', '전월 판매량': '920', '전월 대비 증감': '293', '판매월': '2022-01'}
{'회사명': '볼보', '판매량': '1004', '점유율': '5.9', '전월 판매량': '1418', '전월 대비 증감': '-414', '판매월': '2022-01'}
2022-02
{'회사명': '벤츠', '판매량': '5970', '점유율': '31.2', '전월 판매량': '3405', '전월 대비 증감': '2565', '판매월': '2022-02'}
{'회사명': 'BMW', '판매량': '5656', '점유율': '29.6', '전월 판매량': '5550', '전월 대비 증감': '106', '판매월': '2022-02'}
{'회사명': '아우디', '판매량': '1227', '점유율': '6.4', '전월 판매량': '1269', '전월 대비 증감': '-42', '판매월': '2022-02'}
{'회사명': '폭스바겐', '판매량': '1108', '점유율': '5.8', '전월 판매량': '1213', '전월 대비 증감': '-105', '판매월': '2022-02'}
{'회사명': '볼보', '판매량': '1047', '점유율': '5.5', '전월 판매량': '1004', '전월 대비 증감': '43', '판매월

In [7]:
df = pd.DataFrame(data_list)
df

,회사명,판매량,점유율,전월 판매량,전월 대비 증감,판매월
0,BMW,5550,32.8,4233,1317,2022-01
1,벤츠,3405,20.1,6752,-3347,2022-01
2,아우디,1269,7.5,4373,-3104,2022-01
3,폭스바겐,1213,7.2,920,293,2022-01
4,볼보,1004,5.9,1418,-414,2022-01
...,...,...,...,...,...,...
150,BMW,6380,29.2,6172,208,2024-07
151,벤츠,4369,20.0,6661,-2292,2024-07
152,테슬라,2680,12.3,5293,-2613,2024-07
153,볼보,1411,6.5,1452,-41,2024-07


In [8]:
import mysql.connector

# 데이터베이스 연결 설정
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': '1234',
    'database': 'pjt01'
}


# 데이터베이스 연결
try:
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor(buffered=True)

    for entry in data_list:
        company_name = entry['회사명']
        
        # 모델명을 통해 vehicle_id 가져오기
        cursor.execute("SELECT company_id FROM company WHERE company_name = %s", (company_name,))
        company_id = cursor.fetchone()
        
        if company_id:
            company_id = company_id[0]  # 첫 번째 요소 가져오기
            
            # 데이터 삽입
            cursor.execute("""
                INSERT INTO international_company_sales (sales, market_share, last_month_sales, last_month_change, sales_month, company_id) 
                VALUES (%s, %s, %s, %s, %s, %s)
            """, (entry['판매량'], entry['점유율'], entry['전월 판매량'], entry['전월 대비 증감'], entry['판매월']+'-01', company_id))
        else:
            print(f"{company_name}에 대한 company_id를 찾을 수 없습니다.")

    # 변경 사항 커밋
    conn.commit()
    print("데이터가 성공적으로 삽입되었습니다.")

except mysql.connector.Error as err:
    print(f"오류 발생: {err}")

finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()


데이터가 성공적으로 삽입되었습니다.


In [9]:
# 국내 판매량

data_list = []

for i in range(2022,2025):
    # 2022 ~ 2024
    for j in range(1,13):
        # 2022-01 ~ 2024-08까지만 출력
        if i == 2024 and j == 9:
            break;
        
        wd.get(f'https://auto.danawa.com/auto/?Work=record&Tab=Grand&Month={i}-{j:02}-00')
        time.sleep(2)
        
        element = wd.find_element(By.CSS_SELECTOR, '#autodanawa_gridC > div.gridMain > article > main > div:nth-child(1) > table > tbody')
        rows = element.find_elements(By.TAG_NAME, 'tr')
        
        sales_month = f'{i}-{j:02}'
        print(sales_month)

        for row in rows:
            # 다른 class 속성이 있는 경우 건너뜀 - <tr>에 대해서만 작동
            class_attribute = row.get_attribute('class')
            if class_attribute and ('sub' in class_attribute or 'model_' in class_attribute):
                continue  
            
            # 회사명
            company_element = row.find_element(By.CSS_SELECTOR, 'td.title a')
            company_name = company_element.text.strip()  # 회사명
            
            # 판매량
            sales = row.find_element(By.CLASS_NAME, 'num').text.split(' ')[0].replace(',', '')
            
            # 점유율
            market_share = row.find_element(By.CLASS_NAME, 'rate').text.replace('%', '')
            
            # 전월 대비 판매량 추출
            format_change = lambda text: '-' + text.split('▼')[0].strip() if '▼' in text else text.split('▲')[0].strip()
            last_month_values = row.find_elements(By.CLASS_NAME, 'right')[1].text.split('\n')
            
            last_month_sales = last_month_values[0].replace(',', '') # 전월 판매량
            last_month_change = format_change(last_month_values[1]).replace(',', '') # 전월 대비 증감
            
            data_dict = {
                '회사명': company_name,
                '판매량': sales,
                '점유율': market_share,
                '전월 판매량': last_month_sales,
                '전월 대비 증감': last_month_change,
                '판매월' : sales_month
            }
            print(data_dict)
            data_list.append(data_dict)

2022-01
{'회사명': '기아', '판매량': '37038', '점유율': '39.4', '전월 판매량': '47789', '전월 대비 증감': '-10751', '판매월': '2022-01'}
{'회사명': '현대', '판매량': '35625', '점유율': '37.9', '전월 판매량': '51240', '전월 대비 증감': '-15615', '판매월': '2022-01'}
{'회사명': '제네시스', '판매량': '10575', '점유율': '11.3', '전월 판매량': '14872', '전월 대비 증감': '-4297', '판매월': '2022-01'}
{'회사명': 'KGM', '판매량': '4836', '점유율': '5.2', '전월 판매량': '5810', '전월 대비 증감': '-974', '판매월': '2022-01'}
{'회사명': '르노코리아', '판매량': '4477', '점유율': '4.8', '전월 판매량': '7162', '전월 대비 증감': '-2685', '판매월': '2022-01'}
{'회사명': '쉐보레', '판매량': '1344', '점유율': '1.4', '전월 판매량': '2519', '전월 대비 증감': '-1175', '판매월': '2022-01'}
2022-02
{'회사명': '현대', '판매량': '41994', '점유율': '40.7', '전월 판매량': '35625', '전월 대비 증감': '6369', '판매월': '2022-02'}
{'회사명': '기아', '판매량': '39560', '점유율': '38.3', '전월 판매량': '37038', '전월 대비 증감': '2522', '판매월': '2022-02'}
{'회사명': '제네시스', '판매량': '11016', '점유율': '10.7', '전월 판매량': '10575', '전월 대비 증감': '441', '판매월': '2022-02'}
{'회사명': 'KGM', '판매량': '4540', '점유율': '4.4', '전월 판매량': '4836'

In [10]:
df = pd.DataFrame(data_list)
df

,회사명,판매량,점유율,전월 판매량,전월 대비 증감,판매월
0,기아,37038,39.4,47789,-10751,2022-01
1,현대,35625,37.9,51240,-15615,2022-01
2,제네시스,10575,11.3,14872,-4297,2022-01
3,KGM,4836,5.2,5810,-974,2022-01
4,르노코리아,4477,4.8,7162,-2685,2022-01
...,...,...,...,...,...,...
187,기아,40685,38.5,46238,-5553,2024-08
188,제네시스,10323,9.8,10703,-380,2024-08
189,KGM,3943,3.7,4237,-294,2024-08
190,쉐보레,1587,1.5,2164,-577,2024-08


In [11]:
import mysql.connector

# 데이터베이스 연결 설정
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': '1234',
    'database': 'pjt01'
}


# 데이터베이스 연결
try:
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor(buffered=True)

    for entry in data_list:
        company_name = entry['회사명']
        
        # 모델명을 통해 vehicle_id 가져오기
        cursor.execute("SELECT company_id FROM company WHERE company_name = %s", (company_name,))
        company_id = cursor.fetchone()
        
        if company_id:
            company_id = company_id[0]  # 첫 번째 요소 가져오기
            
            # 데이터 삽입
            cursor.execute("""
                INSERT INTO domestic_company_sales (sales, market_share, last_month_sales, last_month_change, sales_month, company_id) 
                VALUES (%s, %s, %s, %s, %s, %s)
            """, (entry['판매량'], entry['점유율'], entry['전월 판매량'], entry['전월 대비 증감'], entry['판매월']+'-01', company_id))
        else:
            print(f"{company_name}에 대한 company_id를 찾을 수 없습니다.")

    # 변경 사항 커밋
    conn.commit()
    print("데이터가 성공적으로 삽입되었습니다.")

except mysql.connector.Error as err:
    print(f"오류 발생: {err}")

finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()


데이터가 성공적으로 삽입되었습니다.
